In [1]:
import numpy as np
import re
import pandas as pd
from tqdm.notebook import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from typing import List, Optional
import enum

from langchain_community.llms import Ollama
from langchain.output_parsers.regex_dict import RegexDictParser
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage, SystemMessage, ChatMessage
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from pydantic import BaseModel, Field, validator, create_model
from openai import AsyncOpenAI, OpenAI
#import asyncio
import os
from typing import Tuple, List, Optional

import requests

from pydantic import BaseModel, ValidationInfo, model_validator

import json

import itertools
from copy import deepcopy
from tqdm.notebook import tqdm, trange
from sklearn.cluster import KMeans

import umap.umap_ as umap
#import umap
import hdbscan

from src.bubble import *
from src.models import *
from src.utilities import *

Retrieved company Darty : 1707313014508x102198350946437700
Retrieved project Darty_trustpilot : 1707329196900x870734705097005300


In [179]:
types_df = get("Type", constraints=[])
categories_df = get("Category")
subcategories_df = get("SubCategory")
aspects_df = get("Aspect")

In [10]:
aspects_df.to_csv("aspects_df.csv")

In [192]:
ID_CATEG_NONE = categories_df[categories_df["Name"].isna()].iloc[0]["_id"]
REAL_CATEGORIES = categories_df[categories_df["_id"] != ID_CATEG_NONE]
ID_SUBCATEGS_NONE = list(subcategories_df[subcategories_df["Name"].isna()]["_id"])


In [189]:
import pandas as pd


def analyze_feedback_by_period(df, company, category, subcategory, period):
  """Analyzes customer feedback by company, category, subcategory, and period.

  Args:
      df: A pandas DataFrame containing the customer feedback data.
      company: The company to filter by.
      category: The category to filter by.
      subcategory: The subcategory to filter by.
      period: The desired period for grouping the data (e.g., "D" for day, "W" for week, "M" for month, "Y" for year).

  Returns:
      A pandas DataFrame with the following columns:
          * Date: The index of the DataFrame, containing the dates formatted according to the period, 
                  starting from the minimum date of the entire DataFrame.
          * count: The number of customer feedback entries for each period (0 if no entries).
          * ... (similar columns for rating counts, mean, and standard deviation)
          
      The min and std columns will have None values for periods with no entries.
  """

  # Get the minimum date from the entire DataFrame
  min_date = df["Date"].min()

  # Filter the DataFrame by the specified criteria

  filter = (df["Company"] == company)
  if category != ID_CATEG_NONE:
    filter &= (df["Category"] == category)
    if subcategory not in ID_SUBCATEGS_NONE:
      filter &= (df["SubCategory"] == subcategory)

  filtered_df = df[filter]

  # Handle cases where no data matches the filter
  if filtered_df.empty:
    res_df = pd.DataFrame({
        "Date": pd.date_range(start=min_date, periods=0, freq=period)})
    res_df["count"]= 0
    res_df["rating_1"]= 0
    res_df["rating_2"]= 0
    res_df["rating_3"]= 0
    res_df["rating_4"]= 0
    res_df["rating_5"]= 0
    res_df["min"]= 0
    res_df["max"]= 0
    res_df["mean"]= 0
    res_df["std"]= 0
    res_df["median"]= 0
    res_df["q1"]= 0
    res_df["q3"]= 0
    res_df["Date Display"] = res_df["Date"].apply(
      lambda x: format_date(x, period)
  )
    return res_df
  
  # Convert the "Date" column to datetime format
  filtered_df["Date"] = pd.to_datetime(filtered_df["Date"])

  # Define date format function based on period
  def format_date(date, period):
    if period == "D":
      return date.strftime("%d/%m/%Y")
    elif period == "W":
      return date.strftime("%U-%Y")  # Week format with year, e.g., 01-2024
    elif period == "M":
      return date.strftime("%m/%Y")
    else:
      return date.strftime("%Y")

  # Resample the data by the specified period, aggregating on various columns
  grouped_data = (
      filtered_df.set_index("Date")
      .resample(period)
      .agg(
          count=("Rating", "count"),
          rating_1=("Rating", lambda x: x[x == 1].count()),
          rating_2=("Rating", lambda x: x[x == 2].count()),
          rating_3=("Rating", lambda x: x[x == 3].count()),
          rating_4=("Rating", lambda x: x[x == 4].count()),
          rating_5=("Rating", lambda x: x[x == 5].count()),
          mean=("Rating", "mean"),
          std=("Rating", "std"),
          median=("Rating", "median"),
          min=("Rating", "min"),
          max=("Rating", "max"),
          q1=("Rating", lambda x: x.quantile(0.25)),
          q3=("Rating", lambda x: x.quantile(0.75)),
      )
      .reset_index()
  )

  
  # Set the minimum date from entire DataFrame as the starting index
  grouped_data.set_index("Date", inplace=True)
  # Create a date range with appropriate length based on min and max dates of filtered data
  date_range = pd.date_range(
      start=min_date, end=filtered_df["Date"].max(), freq=period
  )

  # Reindex to fill missing periods with 0 and None for mean/std
  grouped_data = grouped_data.reindex(date_range).fillna(0)

  # Fill missing values with appropriate values
  grouped_data.loc[:, "count"] = grouped_data["count"].fillna(0)
  grouped_data.loc[grouped_data["count"] == 0, ["mean", "std", "median", "min", "max", "q1", "q3"]] = None

  grouped_data = grouped_data.rename_axis('Date').reset_index()

  # Apply date formatting based on period
  grouped_data["Date Display"] = grouped_data["Date"].apply(
      lambda x: format_date(x, period)
  )
  return grouped_data


In [190]:
def to_list(s):
    return [0 if pd.isna(x) else x for x in s]

def send_aspects(df, company, category, subcategory, period, grouped_by=None):
    d = {
        "Company": company,
        "Project": PROJECT_ID,
        "Category": category,
        "SubCategory": subcategory,
        "Period": period,
        "Grouped by": grouped_by,
        "Counts": to_list(df["count"]),
        "Counts of 1s": to_list(df["rating_1"]),
        "Counts of 2s": to_list(df["rating_2"]),
        "Counts of 3s": to_list(df["rating_3"]),
        "Counts of 4s": to_list(df["rating_4"]),
        "Counts of 5s": to_list(df["rating_5"]),
        "Dates": to_list(df["Date"].apply(lambda x:pd.to_datetime(x).strftime("%m/%d/%Y"))), #
        "Dates Display": to_list(df["Date Display"]),
        "Max Ratings": to_list(df["min"]),
        "Mean Ratings": to_list(df["mean"]),
        "Std Ratings": to_list(df["std"]),
        "Med Ratings": to_list(df["median"]),
        "Min Ratings": to_list(df["max"]),
        "Q1 Rating": to_list(df["q1"]),
        "Q3 Rating": to_list(df["q3"]),
        }  

    bubble_id = bubble_client.create("Aspect Evol",d)
    return bubble_id

In [186]:
aspects_df

,Company,Project,Rating,SubCategory,Associated_feedback,Date,Category,_id,Explanation
0,1707313014508x102198350946437700,1707329196900x870734705097005300,5,1709253069790x305647624075015900,1707643040311x281208184060071520,2024-02-07 00:00:00+00:00,1709253069102x717430988942903300,1709288669731x792381451006279400,NaN
1,1707313014508x102198350946437700,1707329196900x870734705097005300,5,1709253063096x594485916638931600,1707643040311x281208184060071520,2024-02-07 00:00:00+00:00,1709253062423x867774813720628500,1709288669735x628249787027423700,NaN
2,1707313014508x102198350946437700,1707329196900x870734705097005300,5,1709253066544x770991303066428400,1707643040311x281208184060071520,2024-02-07 00:00:00+00:00,1709253065849x444427432726514300,1709288669739x307796151670405300,La touche Touch ID permet de s'authentifier ra...
3,1707313014508x102198350946437700,1707329196900x870734705097005300,3,1709253068464x983179727881815700,1707643040311x281208184060071520,2024-02-07 00:00:00+00:00,1709253065849x444427432726514300,1709288669741x932504918890495000,"Le prix est élevé, comme pour tous les produit..."
4,1707313014508x102198350946437700,1707329196900x870734705097005300,5,1709253077078x822788218820358100,1707643040316x844188021545895200,2024-01-31 00:00:00+00:00,1709253075821x830225984700473300,1709288671136x151764047078959840,La prise en charge de l'installation de la mac...
...,...,...,...,...,...,...,...,...,...
2202,1707313014508x102198350946437700,1707329196900x870734705097005300,1,1709253064382x227766121950602430,1707643051258x374538859254433860,2023-10-21 00:00:00+00:00,1709253062423x867774813720628500,1709290003979x187358384636157500,Le conseiller n'a pas su proposer une solution...
2203,1707313014508x102198350946437700,1707329196900x870734705097005300,5,1709253076447x308401889339217700,1707643051261x530659398374961660,2023-10-05 00:00:00+00:00,1709253075821x830225984700473300,1709290005459x931272324303185500,NaN
2204,1707313014508x102198350946437700,1707329196900x870734705097005300,5,1709253066544x770991303066428400,1707643051261x530659398374961660,2023-10-05 00:00:00+00:00,1709253065849x444427432726514300,1709290005472x763865961084115600,NaN
2205,1707313014508x102198350946437700,1707329196900x870734705097005300,1,1709253078419x303428894799921100,1707643051261x530659398374961660,2023-10-05 00:00:00+00:00,1709253075821x830225984700473300,1709290005476x395899141293267300,Le vendeur a induit le client en erreur concer...


In [175]:
subcategories_df

,Company,Name,Project,Category,_id
0,1707313014508x102198350946437700,Accueil et assistance,1707329196900x870734705097005300,1709253062423x867774813720628500,1709253063096x594485916638931600
1,1707313014508x102198350946437700,Temps d'attente,1707329196900x870734705097005300,1709253062423x867774813720628500,1709253063739x664374084881986800
2,1707313014508x102198350946437700,Résolution de problèmes,1707329196900x870734705097005300,1709253062423x867774813720628500,1709253064382x227766121950602430
3,1707313014508x102198350946437700,Communication,1707329196900x870734705097005300,1709253062423x867774813720628500,1709253065034x409828009667640600
4,1707313014508x102198350946437700,Qualité,1707329196900x870734705097005300,1709253065849x444427432726514300,1709253066544x770991303066428400
5,1707313014508x102198350946437700,Disponibilité,1707329196900x870734705097005300,1709253065849x444427432726514300,1709253067173x591291065680345900
6,1707313014508x102198350946437700,Diversité,1707329196900x870734705097005300,1709253065849x444427432726514300,1709253067809x102834735303147800
7,1707313014508x102198350946437700,Prix,1707329196900x870734705097005300,1709253065849x444427432726514300,1709253068464x983179727881815700
8,1707313014508x102198350946437700,Aménagement,1707329196900x870734705097005300,1709253069102x717430988942903300,1709253069790x305647624075015900
9,1707313014508x102198350946437700,Propreté,1707329196900x870734705097005300,1709253069102x717430988942903300,1709253070437x213899894680302560


In [193]:
for category in tqdm(categories_df["_id"]):
    subcategories = subcategories_df[subcategories_df["Category"]==category]["_id"]

    for subcategory in subcategories:
        for period in ["M"]: #["D","W", "M", "Y"]:
            df = analyze_feedback_by_period(aspects_df, COMPANY_ID, category, subcategory, period)
            send_aspects(df, COMPANY_ID, category, subcategory, period)

  0%|          | 0/6 [00:00<?, ?it/s]

/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_39802/2947707909.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["Date"] = pd.to_datetime(filtered_df["Date"])
/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_39802/2947707909.py:76: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(period)
/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_39802/2947707909.py:99: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  date_range = pd.date_range(
/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_39802/2947707909.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [ ]:
df[[f"Count of {i}s" for i in range(1,6)]].plot(kind='bar', stacked=True)
